## Day 9: Rope Bridge

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```

In [1]:
import pytest
from icecream import ic

In [2]:
def load_input(filename):
    with open(filename, 'r') as f_input:
        for line in f_input:
            move, distance = line.strip().split()
            yield move, int(distance)  

In [3]:
assert list(load_input('sample.txt')) == [
    ('R', 4),
    ('U', 4),
    ('L', 3),
    ('D', 1),
    ('R', 4),    
    ('D', 1),
    ('L', 5),
    ('R', 2),
    ]

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

```
..##..
...##.
.####.
....#.
s###..
```

In [4]:
# We'll need a signum fucntion

def signum(i: int) -> int:
    return 0 if i == 0 else 1 if i > 0 else -1

assert signum(-6) == -1
assert signum(-1) == -1
assert signum(0) == 0
assert signum(1) == 1
assert signum(17) == 1

In [5]:
from dataclasses import dataclass

@dataclass(eq=True, frozen=True)
class Vector2:
    x: int = 0
    y: int = 0
    
    def __sub__(self, other):
        return Vector2(
            other.x - self.x,
            other.y - self.y,
        )

    def __add__(self, other):
        return Vector2(
            other.x + self.x,
            other.y + self.y,
        )

    
    def as_signum(self):
        return Vector2(
            signum(self.x),
            signum(self.y),
        )

In [6]:
assert Vector2(2,2) - Vector2(7, -1) == Vector2(5, -3)
assert Vector2(2,2) + Vector2(7, -1) == Vector2(9, 1)

# Vector2 is hashable, so it can be included in a set
s = set([Vector2(), Vector2(1,1), Vector2(1,1)])
assert len(s) == 2

In [7]:
def delta(v0: Vector2, v1: Vector2) -> Vector2:
    diff = v0 - v1
    return Vector2(signum(diff.x), signum(diff.y))


assert delta(Vector2(2,2), Vector2(7, -1)) == Vector2(1, -1)
assert delta(Vector2(2,2), Vector2(7, -1)) == Vector2(1, -1)
assert delta(Vector2(2,2), Vector2(2, 2)) == Vector2(0, 0)

In [8]:
# Let's move it

def move(v: Vector2, movement):
    match movement:
        case 'U': return Vector2(v.x, v.y + 1)
        case 'D': return Vector2(v.x, v.y - 1)
        case 'L': return Vector2(v.x - 1, v.y)
        case 'R': return Vector2(v.x + 1, v.y)
        case _: raise ValueError(f"Expected [U|D|L|R], got {_}")

In [9]:
v = Vector2(6, 9)
assert move(v, 'U') == Vector2(6, 10)
assert move(v, 'D') == Vector2(6, 8)
assert move(v, 'L') == Vector2(5, 9)
assert move(v, 'R') == Vector2(7, 9)

In [10]:
def too_far(vec: Vector2) -> bool:
    return max([abs(vec.x), abs(vec.y)]) > 1

In [11]:
# max distance 0
assert too_far(Vector2(0, 0) - Vector2(-1, -1)) is False

# max distance 1
assert too_far(Vector2(0, 0) - Vector2(0, -1)) is False
assert too_far(Vector2(0, 0) - Vector2(1, -1)) is False
assert too_far(Vector2(0, 0) - Vector2(-1, 0)) is False
assert too_far(Vector2(0, 0) - Vector2(0, 0)) is False
assert too_far(Vector2(0, 0) - Vector2(1, 0)) is False
assert too_far(Vector2(0, 0) - Vector2(-1, 1)) is False
assert too_far(Vector2(0, 0) - Vector2(0, 1)) is False
assert too_far(Vector2(0, 0) - Vector2(1, 1)) is False

# max distance 2
assert too_far(Vector2(0, 0) - Vector2(-2, -2)) is True
assert too_far(Vector2(0, 0) - Vector2(-1, -2)) is True
assert too_far(Vector2(0, 0) - Vector2(0, -2)) is True
assert too_far(Vector2(0, 0) - Vector2(1, -2)) is True
assert too_far(Vector2(0, 0) - Vector2(2, -2)) is True

assert too_far(Vector2(0, 0) - Vector2(-2, -1)) is True
assert too_far(Vector2(0, 0) - Vector2(-1, -1)) is False
assert too_far(Vector2(0, 0) - Vector2(0, -1)) is False
assert too_far(Vector2(0, 0) - Vector2(1, -1)) is False
assert too_far(Vector2(0, 0) - Vector2(2, -1)) is True

assert too_far(Vector2(0, 0) - Vector2(-2, 0)) is True
assert too_far(Vector2(0, 0) - Vector2(-1, 0)) is False
assert too_far(Vector2(0, 0) - Vector2(0, 0)) is False
assert too_far(Vector2(0, 0) - Vector2(1, 0)) is False
assert too_far(Vector2(0, 0) - Vector2(2, 0)) is True

assert too_far(Vector2(0, 0) - Vector2(-2, 1)) is True
assert too_far(Vector2(0, 0) - Vector2(-1, 1)) is False
assert too_far(Vector2(0, 0) - Vector2(0, 1)) is False
assert too_far(Vector2(0, 0) - Vector2(1, 1)) is False
assert too_far(Vector2(0, 0) - Vector2(2, 1)) is True

assert too_far(Vector2(0, 0) - Vector2(-2, 2)) is True
assert too_far(Vector2(0, 0) - Vector2(-1, 2)) is True
assert too_far(Vector2(0, 0) - Vector2(0, 2)) is True
assert too_far(Vector2(0, 0) - Vector2(1, 2)) is True
assert too_far(Vector2(0, 0) - Vector2(2, 2)) is True

In [12]:
def solution_one(filename):
    head = Vector2()
    tail = Vector2()
    visited = set([tail])
    for movement, steps in load_input(filename):
        for step in range(steps):
            head = move(head, movement)
            distance = tail - head
            if too_far(distance):
                tail = tail + distance.as_signum()
                visited.add(tail)
    return len(visited)

So, there are **13** positions the tail visited at least once.

In [13]:
assert solution_one('sample.txt') == 13

Simulate your complete hypothetical series of motions. **How many positions does the tail of the rope visit at least once?**

In [14]:
sol = solution_one('input.txt')
print(f"Solution part one: {sol}")

Solution part one: 6018


## Part Two

A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:

(some text omitted, see the full text in [Rope Bridge](https://adventofcode.com/2022/day/9)

In [15]:
def dump(title, head, tail, visited):
    min_y = min([p.y for p in visited] + [head.y] + [p.y for p in tail])
    max_y = max([p.y for p in visited] + [head.y] + [p.y for p in tail])
    min_x = min([p.x for p in visited] + [head.x] + [p.x for p in tail])
    max_x = max([p.x for p in visited] + [head.x] + [p.x for p in tail])
    print(f"--[ {title} ]-------------------------")
    for y in range(max_y, min_y-1, -1):
        for x in range(min_x, max_x+1):
            v = Vector2(x, y)
            if v in tail:
                print(1 + tail.index(v), end='')
            elif v in visited:
                print('#', end='')
            elif v == head:
                print('H', end='')
            else:
                print('.', end='')
        print()

In [16]:
def solution_two(filename, tron=False):
    head = Vector2()
    tail = [Vector2()] * 9
    visited = set([tail[-1]])
    for movement, steps in load_input(filename):
        for step in range(steps):
            head = move(head, movement)
            target = head
            for index in range(9):
                distance = tail[index] - target
                if too_far(distance):
                    tail[index] += distance.as_signum()
                target = tail[index]
            visited.add(target)
        # dump(f"{movement} {steps}", head, tail, visited) 
    if tron:
        dump('Final state', head, tail, visited)
    return len(visited)

In [17]:
assert solution_two('sample_2.txt', tron=True) == 36

--[ Final state ]-------------------------
H.....................
1.....................
2.....................
3.....................
4.....................
5.....................
6.....................
7.....................
8.....................
9.....................
#.............###.....
#............#...#....
.#..........#.....#...
..#..........#.....#..
...#........#.......#.
....#......#.........#
.....#..............#.
......#............#..
.......#..........#...
........#........#....
.........########.....


In [18]:
sol = solution_two('input.txt')
print(f"Solution part two: {sol}")

Solution part two: 2619
